In [77]:
import keras
from keras.layers import Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Model
from keras.layers import Input
from keras.layers import BatchNormalization
from keras.layers import UpSampling2D
from keras.layers import Concatenate

In [78]:
import scipy.io as scio
import numpy as np    
import os
import matplotlib.pyplot as plt
from scipy.misc import imsave
from scipy import ndimage, misc

In [79]:
images = []
for root, dirnames, filenames in os.walk("Dataset/Train_images/"):
    for filename in filenames:
        filepath = os.path.join(root, filename)
        image = ndimage.imread(filepath, mode="L")
        images.append(image)

In [80]:
print (len(image))


496


In [81]:
labels = np.load('train_labels.npy')
labels_list = []
for i in range(len(labels)):
    labels_list.append(labels[i])

In [97]:
print (len(labels_list))
print (type(labels_list[0]))

images=np.array(images)
images = images.reshape(images.shape[0],496,96,1)

print(images[0].shape)
train_labels=labels_list[0:500]
train_images=images[0:500]
test_labels = labels_list[500:550]
test_images = images[500:550]

550
<type 'numpy.ndarray'>
(496, 96, 1)


In [98]:
print (len(train_images))
print (len(train_labels))
print (np.array(train_labels).shape)

500
500
(500, 496, 96, 8)


In [90]:
# Defines the input tensor
inputs = Input(shape=(496,96,1))

L1 = Conv2D(64,kernel_size=(7,3),activation = 'relu',padding = "same")(inputs)
L2 = BatchNormalization()(L1)
L3 = MaxPooling2D(pool_size=(2,2))(L2)
L4 = Conv2D(64,kernel_size=(7,3),activation = 'relu',padding = "same")(L3)
L5 = BatchNormalization()(L4)
L6 = MaxPooling2D(pool_size=(2,2))(L5)
L7 = Conv2D(64,kernel_size=(7,3),activation = 'relu',padding = "same")(L6)
L8 = BatchNormalization()(L7)
L9 = MaxPooling2D(pool_size=(2,2))(L8)
L10 = Conv2D(64,kernel_size=(7,3),activation = 'relu',padding = "same")(L9)
L11 = BatchNormalization()(L10)
L12 = UpSampling2D(size = (2,2))(L11)
L13 = Concatenate(axis = 3)([L8,L12])
L14 = Conv2D(64,kernel_size=(7,3),activation = 'relu',padding = "same")(L13)
L15 = BatchNormalization()(L14)
L16 = UpSampling2D(size= (2,2))(L15)
L17 = Concatenate(axis = 3)([L16,L5])
L18 = Conv2D(64,kernel_size=(7,3),activation = 'relu',padding = "same")(L17)
L19 = BatchNormalization()(L18)
L20 = UpSampling2D(size=(2,2))(L19)
L21 = Concatenate(axis=3)([L20,L2])
L22 = Conv2D(64,kernel_size=(7,3),activation = 'relu',padding = "same")(L21)
L23 = BatchNormalization()(L22)
L24 = Conv2D(8,kernel_size=(1,1),activation = 'softmax',padding = "same")(L23)

model = Model(inputs = inputs, outputs = L24)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 496, 96, 1)        0         
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 496, 96, 64)       1408      
_________________________________________________________________
batch_normalization_42 (Batc (None, 496, 96, 64)       256       
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 248, 48, 64)       0         
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 248, 48, 64)       86080     
_________________________________________________________________
batch_normalization_43 (Batc (None, 248, 48, 64)       256       
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 124, 24, 64)       0         
__________

In [91]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
train_images_2=np.array(train_images)
train_labels_2=np.array(train_labels)
model.fit(train_images_2,train_labels_2,batch_size=6,epochs=10)

Epoch 1/10
 90/500 [====>.........................] - ETA: 260s - loss: 1.9176 - acc: 0.5939